In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_csv(r'C:\Users\ywang\Documents\R\OnboardSurvey_2020Oct_yq\Data and Reports\Petaluma\2018\As CSV\20180530_OD_Petaluma_Submittal_FINAL NO POUND NO SINGLE QUOTE.csv')
print('read {} rows with {} unique id'.format(df_raw.shape[0], len(df_raw.id.unique())))

read 286 rows with 286 unique id


In [3]:
df = df_raw.copy()

# the following columns were named wrong, correct them
df.rename(columns = {'final_change_to_access_mode.1' : 'final_suggested_access_mode',
                     'final_change_to_egress_mode.1' : 'final_suggested_egress_mode'}, inplace=True)

In [4]:
# consolidate 'fare_cat' and 'fare_cat_other'

df['fare_type'] = df['kind_of_fare']
df.loc[df.fare_type == 'Other', 'fare_type'] = df['kind_of_fare_other']

In [5]:
# create a new field to show detailed fare_medium

display(df[['fare_medium', 'clipper_detail', 'pm1_clipper _other', 
            'pm2_cash_or_paper', 'pm3_by_other', 'pm3_by_other_other']].drop_duplicates())

df['fare_medium_detail'] = df['fare_medium']

df.loc[df.pm3_by_other == 'Other', 'pm3_by_other'] = df['pm3_by_other_other']
df.loc[df.fare_medium_detail == 'BY OTHER ', 'fare_medium_detail'] = df['pm3_by_other']

df.loc[df.fare_medium_detail == 'BY CLIPPER CARD', 'fare_medium_detail'] = df['clipper_detail']

df.loc[df.fare_medium_detail == 'BY CASH OR PAPER ', 'fare_medium_detail'] = df['pm2_cash_or_paper']

,fare_medium,clipper_detail,pm1_clipper _other,pm2_cash_or_paper,pm3_by_other,pm3_by_other_other
0,BY OTHER,NaN,NaN,NaN,Paratransit Free Rider,NaN
1,BY OTHER,NaN,NaN,NaN,Free Ride Token,NaN
5,BY CLIPPER CARD,Adult 31-day,NaN,NaN,NaN,NaN
6,BY CLIPPER CARD,Clipper Cash,NaN,NaN,NaN,NaN
9,BY CASH OR PAPER,NaN,NaN,Cash (coins and bills),NaN,NaN
12,BY CASH OR PAPER,NaN,NaN,Adult Monthly,NaN,NaN
15,BY OTHER,NaN,NaN,NaN,SRJC Student Pass,NaN
25,BY CASH OR PAPER,NaN,NaN,Half Price Ticket,NaN,NaN
26,BY CASH OR PAPER,NaN,NaN,Transfer,NaN,NaN
30,BY CASH OR PAPER,NaN,NaN,Half Price Monthly,NaN,NaN


In [6]:
# consolidate 'gender' and 'gender_other'
df['gender_detail'] = df['gender']
df.loc[df.gender_detail == 'Other', 'gender_detail'] = df['gender_other']

In [7]:
# consolidate 'language_at_home_detail' and 'home_other_lang _other'
df.loc[df.language_at_home_detail == 'Other', 'language_at_home_detail'] = df['home_other_lang _other']

In [8]:
# print(list(df))
df.to_csv(r'C:\Users\ywang\Documents\R\OnboardSurvey_2020Oct_yq\Data and Reports\Petaluma\2018\As CSV\20180530_OD_Petaluma_Submittal_addCols_FINAL NO POUND NO SINGLE QUOTE.csv', index=False)

In [9]:
# bring in standard dictionary to check field consistency

# dictionary for Petaluma survey
var = pd.read_csv(r'C:\Users\ywang\Documents\R\OnboardSurvey_2020Oct_yq\Data and Reports\Petaluma\2018\As CSV\variables_raw.csv',
                  encoding = "ISO-8859-1", engine='python')

# standard dictionary
var_standard = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\onboard-surveys\make-uniform\production\Dictionary for Standard Database.csv')
var_standard.columns = [x+'_s' for x in list(var_standard)]

# merge
var_merge = var.merge(var_standard, left_on='Generic_Variable', right_on='Generic_Variable_s', how='outer')

# check if 'Generic_Variable' in Petaluma dictionary matches the standard 'Generic_Variable'. chk1 should be empty
chk1 = var_merge.loc[(var_merge.Generic_Variable.notnull()) & (var_merge.Generic_Variable_s.isnull())]
print('Generic_Variable that should not exit:')
print(chk1.Generic_Variable.unique())
print()

# check if columns names in survey data matches 'Survey_Variable' in Petaluma dictionary.
# the following loops should not include variables that are needed for standardization

print('variables not in standard dictionary')
for i in var.loc[var.Generic_Variable.notnull()]['Survey_Variable']:
    if i not in list(df):
        print(i)

print('variables in df but not in the current Marin Transit dictionary')        
for i in list(df):
    if i not in list(var.Survey_Variable):
        print(i)

Generic_Variable that should not exit:
[]

variables not in standard dictionary
variables in df but not in the current Marin Transit dictionary
survey_type
completed_time
route_surveyed_code
route
change_to_route_surveyed
reason_for_change_to_route_surveyed
final_route_surveyed_code
resident_or_visitor_code
resident_or_visitor
vis_zip
home_address
home_city
home_state
home_zip
hotel_address
hotel_city
hotel_state
hotel_zip
hotel_lat
hotel_lon
origin_place_type_code
origin_place_type
origin_place_type _other
change_to_origin_type_place
reason_for_change_to_origin_type_place
suggestion_for_change_to_origin_type_place_code
suggestion_for_change_to_origin_type_place
suggestion_for_change_to_origin_type_place_other
final_change_to_origin_type_place
final_reason_for_change_to_origin_type_place
final_suggested_origin_type_place_code
origin_address
origin_city
origin_state
origin_zip
orig_lat
orig_lon
change_to_origin_address
reason_for_change_to_origin_address
final_origin_address
final_origi

In [10]:
# check if all the values in the survey data are included in Petaluma dictionary
# look at non-categorical variables; "diff" should be empty or only contains nan

var_clean = var[['operator', 'Survey_year', 'Survey_Variable', 'Survey_Response', 
                 'Generic_Variable', 'Generic_Response']].drop_duplicates()
var_clean = var_clean.loc[var_clean.Generic_Variable.notnull()]

for i in var_clean.loc[var_clean.Survey_Response != 'NONCATEGORICAL']['Survey_Variable'].unique():
    print(i)
    df_sub = df[['id', i]]
    var_sub = var_clean.loc[var_clean.Survey_Variable == i]
    
#     if i in ['Survey_route', 'Direction', 'Origin', 'Destination', 'Access_mode', 'Transfer_from_amount', 
#              'Transfer_to_amount', 'Egress_mode', 'Employment_status', 'Work_before_trip', 'Work_after_trip',
#              'Student_status', 'Been_2school_today', 'Will_go2school_today', 'Time_left_home', 'Time_return_home',
#              'Pay_mode', 'Fare_type', 'Persons_HH', 'Workers_HH', 'Vehicles_HH', 'Hispanic', 'Race',
#              'Lang_other_than_english', 'Other_lang_spoken_at_home', 'English_fluency', 'HH_income', 'Gender',
#              'Survey_language']:
#         var_sub.Survey_Response = var_sub.Survey_Response.apply(lambda x: int(x))

#     if i in ['Access_mode_final', 'Egress_mode_final']:
#         df_sub[i] = df_sub[i].apply(lambda x: str(x))   
        
    compare = df_sub.merge(var_sub, left_on=i, right_on='Survey_Response', how='left')
    diff = compare.loc[compare.Generic_Response.isnull()]
    if diff.shape[0] > 0:
        print(diff[i].unique())

day_type
orig_purp
final_suggested_prev_transfers
final_suggested_access_mode
dest_purp
final_suggested_next_transfers
final_suggested_egress_mode
time_on
fare_type
fare_medium_detail
vehicles
persons
count_employed_hh
work_status
[nan]
at_work_prior_to_orig_purp
[nan]
at_work_after_dest_purp
[nan]
student_status
at_school_prior_to_orig_purp
[nan]
at_school_after_dest_purp
[nan]
race_dmy_ltn
race_dmy_blk
race_dmy_asn
race_dmy_amcn_ind
race_dmy_hwi
race_dmy_whi
gender_detail
household_income
language_at_home_binary
language_at_home_detail
[nan]
eng_proficient
[nan]


In [11]:
# finally, check all necessary fields are included, and export
print(var_clean.Generic_Variable.unique())
var_clean.to_csv(r'C:\Users\ywang\Documents\R\OnboardSurvey_2020Oct_yq\Data and Reports\Petaluma\2018\As CSV\vars_for_standard_dictionary.csv', index=False)

['ID' 'weekpart' 'date_string' 'route' 'home_lat' 'home_lon' 'orig_purp'
 'orig_lat' 'orig_lon' 'number_transfers_orig_board'
 'first_route_before_survey_board' 'second_route_before_survey_board'
 'access_mode' 'dest_purp' 'dest_lat' 'dest_lon'
 'number_transfers_alight_dest' 'first_route_after_survey_alight'
 'second_route_after_survey_alight' 'egress_mode' 'survey_board_lat'
 'survey_board_lon' 'survey_alight_lat' 'survey_alight_lon' 'time_string'
 'time_period' 'fare_category' 'fare_medium' 'vehicles' 'persons'
 'workers' 'work_status' 'workplace_lat' 'workplace_lon'
 'at_work_prior_to_orig_purp' 'at_work_after_dest_purp' 'student_status'
 'school_lat' 'school_lon' 'at_school_prior_to_orig_purp'
 'at_school_after_dest_purp' 'year_born_four_digit' 'hispanic'
 'race_dmy_blk' 'race_dmy_asn' 'race_dmy_ind' 'race_dmy_hwi'
 'race_dmy_wht' 'race_other_string' 'gender' 'household_income'
 'language_at_home_binary' 'language_at_home_detail' 'eng_proficient'
 'first_board_lat' 'first_board_lo

In [12]:
# create raw transfer routes file
routes = pd.DataFrame(columns = ['survey_name'])
for i in ['final_trip_first_route', 'final_trip_second_route',
          'final_trip_to_first_route', 'final_trip_to_second_route']:
    route_unique = df[[i]]
    route_unique.columns = ['survey_name']
    routes = pd.concat([routes, route_unique])

routes_clean = routes.loc[routes.survey_name.notnull()]
routes_clean.drop_duplicates(inplace=True)
routes_clean['survey'] = 'Petaluma Transit'
routes_clean['survey_year'] = 2018

print(routes_clean.shape)
routes_clean[['survey','survey_year','survey_name']].to_csv(r'C:\Users\ywang\Documents\R\OnboardSurvey_2020Oct_yq\Data and Reports\Petaluma\2018\As CSV\all_routes_raw.csv', index=False)

(18, 3)


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea